In [ ]:
import sys

sys.path.append("../")

In [ ]:
import json

import data.drawings.make_tasks as drawing_tasks

In [ ]:
EXPERIMENT_NAME = "gg_laps_domains"
RANDOM_SEEDS = ["111", "222", "333"]

BASH_FILE = "../run_experiments_clevr.sh"

# DOMAINS = [f"drawings_{domain}" for domain in drawing_tasks.TASK_DOMAINS]
# DOMAINS = [DOMAINS[0]]
# DOMAINS = ["re2"]
DOMAINS = ["clevr"]
print(DOMAINS)

In [ ]:
EXPERIMENT_TYPES = [
    "stitch",
    "oracle",
    "oracle_train_test",
    "stitch_codex",
    "stitch_codex_language",
    "stitch_codex_language_origin_random_test",
]
print(EXPERIMENT_TYPES)

In [ ]:
cmds = []
for domain in DOMAINS:
    for experiment_type in EXPERIMENT_TYPES:
        cmd = (
            f"python run_iterative_experiment.py --experiment_name {EXPERIMENT_NAME} --domain {domain} --experiment_type {experiment_type} "
            + "--stitch_params '{\"iterations\": 10}' "
        )
        if "codex" in experiment_type:
            seeds = RANDOM_SEEDS

            codex_params = {
                "n_tasks_per_prompt": 25,
                "n_samples": 50,
                "n_samples_per_query": 5,
                "final_task_origin": "default",
                "body_task_types": ["programs"],
                "final_task_types": ["programs"],
                "function_name_classes": ["human_readable"],
            }

            if "language" in experiment_type:
                codex_params.update(
                    {
                        "body_task_types": ["language", "programs"],
                        "final_task_types": ["language"],
                    }
                )
            cmd += f"--codex_params '{json.dumps(codex_params)}' "
            cmd += "--use_cached "
        else:
            seeds = [RANDOM_SEEDS[0]]

        cmd += f"--random_seeds {' '.join(seeds)} "
        cmd += "\n"
        cmds.append(cmd)

print(cmds[0])

In [ ]:
with open(BASH_FILE, "w") as f:
    f.writelines(cmds)